In [2]:
import folium
import ee
import geemap
from datetime import datetime, timedelta

# Authenticate your Google account with Earth Engine
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = "anr-41793")

In [3]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [4]:
# Region of interest
point = ee.Geometry.Point(-81.660044, 28.473813)
region = point.buffer(distance=100000)

In [5]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2024-04-01', '2024-06-01') \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))

In [6]:
image = collection.median().clip(region)

In [7]:
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

In [8]:
ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')

In [9]:
map = folium.Map(location=[28.473813, -81.660044], tiles="Cartodb dark_matter", zoom_start=8)

In [10]:
map.add_ee_layer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')

In [11]:
map.add_ee_layer(ndwi, {'min': -1, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI')

In [12]:
folium.LayerControl(collapsed = False).add_to(map)
display(map)